## Import

In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1-py3
!pip3 install konlpy
!JAVA_HOME="C:\Program Files\Java\jdk-13.0.2"
!pip install transformers
!pip install sentencepiece
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
from tqdm import tqdm
import os
import re
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,082 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRel

/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(


## 데이터 로드

In [ ]:
import csv
wf = open("tokenized_data.csv", "w")

# 원티드
for i in (16, 17, 18, 21, 22, 23, 27, 29):
  rf = open(f"wanted_tokenized_data_{i}.csv", "r", encoding='utf-8')
  rdr = csv.reader(rf)
  for line in rdr:
    if len(line) == 0:
        wf.write("9999"+","+"9999"+"\n")
    elif len(line) == 1:
        wf.write(line[0]+","+"9999"+"\n")
    else :
        wf.write(line[0]+","+line[1]+"\n")
  rf.close()
# 잡코리아
for i in (16, 17, 18, 19, 20, 21, 22, 23, 24):
  rf = open(f"jobkorea_tokenized_data_{i}.csv", "r", encoding='utf-8')
  rdr = csv.reader(rf)
  for line in rdr:
    if len(line) == 0:
        wf.write("9999"+","+"9999"+"\n")
    elif len(line) == 1:
        wf.write(line[0]+","+"9999"+"\n")
    else :
        wf.write(line[0]+","+line[1]+"\n")
  rf.close()
# 링커리어
for i in (16, 17, 18, 19, 20, 22, 23, 26):
  rf = open(f"linkareer_tokenized_data_{i}.csv", "r", encoding='utf-8')
  rdr = csv.reader(rf)
  for line in rdr:
    if len(line) == 0:
        wf.write("9999"+","+"9999"+"\n")
    elif len(line) == 1:
        wf.write(line[0]+","+"9999"+"\n")
    else :
        wf.write(line[0]+","+line[1]+"\n")
  rf.close()

# 워크넷
for i in (16, 17, 18, 19, 20, 21, 22, 23):
  rf = open(f"tokenized_data_{i}.csv", "r", encoding='utf-8')
  rdr = csv.reader(rf)
  for line in rdr:
    if len(line) == 0:
        wf.write("9999"+","+"9999"+"\n")
    elif len(line) == 1:
        wf.write(line[0]+","+"9999"+"\n")
    else :
        wf.write(line[0]+","+line[1]+"\n")
  rf.close()


In [ ]:
# 원티드
for i in (0, 1, 2, 3, 4, 5, 6, 7, 24, 8, 9, 11, 12, 14, 15, 25, 28):
  rf = open(f"wanted_tokenized_data_{i}.csv", "r", encoding='cp949')
  rdr = csv.reader(rf)
  for line in rdr:
    wf.write(line[0]+","+line[1]+"\n")
  rf.close()
# 잡코리아
for i in (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15):
  rf = open(f"jobkorea_tokenized_data_{i}.csv", "r", encoding='cp949')
  rdr = csv.reader(rf)
  for line in rdr:
    wf.write(line[0]+","+line[1]+"\n")
  rf.close()
# 링커리어
for i in (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 24, 25):
  rf = open(f"linkareer_tokenized_data_{i}.csv", "r", encoding='cp949')
  rdr = csv.reader(rf)
  for line in rdr:
    wf.write(line[0]+","+line[1]+"\n")
  rf.close()

# 워크넷
for i in (0, 1, 2, 3, 4, 5, 6, 7, 24, 8, 9, 10, 11, 12, 13, 14, 15):
  rf = open(f"tokenized_data_{i}.csv", "r", encoding='cp949')
  rdr = csv.reader(rf)
  for line in rdr:
    wf.write(line[0]+","+line[1]+"\n")
  rf.close()

wf.close()

## 데이터 전처리

In [ ]:
train = pd.read_csv("tokenized_data.csv", names=['src', 'tar'], on_bad_lines='skip')

In [ ]:
train

,src,tar
0,플랜티넷,-
1,은,-
2,2000년,-
3,부터,-
4,약,-
...,...,...
72688,추적,-
72689,NaN,-
72690,배급,-
72691,등,-


In [ ]:
# 리스트로 변경
data = [list(x) for x in train[['src', 'tar']].to_numpy()]

In [ ]:
data[:10]

[['플랜티넷', '-'],
 ['은', '-'],
 ['2000년', '-'],
 ['부터', '-'],
 ['약', '-'],
 ['20년', '-'],
 ['동안', '-'],
 ['축척한', '-'],
 ['유하', '-'],
 ['어', '-']]

In [ ]:
# 라벨 추출 & 딕셔너리 형식으로 저장
label = train['tar'].unique().tolist()
label_dict = {word:i for i, word in enumerate(label)}
label_dict.update({"[PAD]":len(label_dict)})
index_to_ner = {i:j for j, i in label_dict.items()}

In [ ]:
def isRight(d):
  return label_dict[d[1]] not in [18, 24, 25, 26, 27, 30, 31]
data = list(filter(isRight, data))

In [ ]:
data[:10]

[['플랜티넷', '-'],
 ['은', '-'],
 ['2000년', '-'],
 ['부터', '-'],
 ['약', '-'],
 ['20년', '-'],
 ['동안', '-'],
 ['축척한', '-'],
 ['유하', '-'],
 ['어', '-']]

In [ ]:
label = []
for d in data:
  label.append(d[1])
label = list(set(label))
print(label)
label_dict = {word:i for i, word in enumerate(label)}
print(label_dict)
label_dict.update({"[PAD]":len(label_dict)})
print(label_dict)
index_to_ner = {i:j for j, i in label_dict.items()}

[nan, 'B_DB', 'I_PL', 'I_CS', 'B_DT', 'B_CS', 'I_HW', 'B_AW', 'I_SC', 'B_SS', 'B_PL', 'I_NT', '9999', 'I_AW', 'B_SC', 'I_OS', 'I_DT', 'I_SW', 'B_HW', 'B_OS', 'B_SW', '-', 'I_SS', 'I_DB', 'B_NT']
{nan: 0, 'B_DB': 1, 'I_PL': 2, 'I_CS': 3, 'B_DT': 4, 'B_CS': 5, 'I_HW': 6, 'B_AW': 7, 'I_SC': 8, 'B_SS': 9, 'B_PL': 10, 'I_NT': 11, '9999': 12, 'I_AW': 13, 'B_SC': 14, 'I_OS': 15, 'I_DT': 16, 'I_SW': 17, 'B_HW': 18, 'B_OS': 19, 'B_SW': 20, '-': 21, 'I_SS': 22, 'I_DB': 23, 'B_NT': 24}
{nan: 0, 'B_DB': 1, 'I_PL': 2, 'I_CS': 3, 'B_DT': 4, 'B_CS': 5, 'I_HW': 6, 'B_AW': 7, 'I_SC': 8, 'B_SS': 9, 'B_PL': 10, 'I_NT': 11, '9999': 12, 'I_AW': 13, 'B_SC': 14, 'I_OS': 15, 'I_DT': 16, 'I_SW': 17, 'B_HW': 18, 'B_OS': 19, 'B_SW': 20, '-': 21, 'I_SS': 22, 'I_DB': 23, 'B_NT': 24, '[PAD]': 25}


In [ ]:
label_dict

{nan: 0,
 'B_DB': 1,
 'I_PL': 2,
 'I_CS': 3,
 'B_DT': 4,
 'B_CS': 5,
 'I_HW': 6,
 'B_AW': 7,
 'I_SC': 8,
 'B_SS': 9,
 'B_PL': 10,
 'I_NT': 11,
 '9999': 12,
 'I_AW': 13,
 'B_SC': 14,
 'I_OS': 15,
 'I_DT': 16,
 'I_SW': 17,
 'B_HW': 18,
 'B_OS': 19,
 'B_SW': 20,
 '-': 21,
 'I_SS': 22,
 'I_DB': 23,
 'B_NT': 24,
 '[PAD]': 25}

In [ ]:
index_to_ner

{0: nan,
 1: 'B_DB',
 2: 'I_PL',
 3: 'I_CS',
 4: 'B_DT',
 5: 'B_CS',
 6: 'I_HW',
 7: 'B_AW',
 8: 'I_SC',
 9: 'B_SS',
 10: 'B_PL',
 11: 'I_NT',
 12: '9999',
 13: 'I_AW',
 14: 'B_SC',
 15: 'I_OS',
 16: 'I_DT',
 17: 'I_SW',
 18: 'B_HW',
 19: 'B_OS',
 20: 'B_SW',
 21: '-',
 22: 'I_SS',
 23: 'I_DB',
 24: 'B_NT',
 25: '[PAD]'}

In [ ]:
# tups에는 [[단어, 개체], [], ...] 저장
tups = []
for i, j in data:
  tups.append([i, label_dict[j]])

In [ ]:
tups[:50]

[['플랜티넷', 21],
 ['은', 21],
 ['2000년', 21],
 ['부터', 21],
 ['약', 21],
 ['20년', 21],
 ['동안', 21],
 ['축척한', 21],
 ['유하', 21],
 ['어', 21],
 ['디지털', 21],
 ['콘텐츠', 21],
 ['데이터베이스(DB)', 21],
 ['를', 21],
 ['바탕', 21],
 ['으로', 21],
 ['변화', 21],
 ['하', 21],
 ['는', 21],
 ['모바일(AW)', 7],
 ['PC', 13],
 ['환경', 13],
 ['에', 13],
 ['적합한', 13],
 ['새롭', 13],
 ['고', 13],
 ['혁신적', 13],
 ['이', 13],
 ['ㄴ', 13],
 ['필터링', 13],
 ['솔루션', 13],
 ['을', 13],
 ['연구', 13],
 ['하고', 21],
 ['제공', 21],
 ['하', 21],
 ['ㅂ니다', 21],
 ['.', 21],
 ['9999', 12],
 ['주요업무', 21],
 ['9999', 12],
 ['•', 21],
 ['악성코드', 14],
 [nan, 8],
 ['공격도구', 8],
 [nan, 8],
 ['APT', 8],
 ['등', 8],
 ['공격', 8],
 ['기법', 8]]

In [ ]:
#from torch import nan
# sentences에는 단어, targets에는 개체 저장 및 문장 구분
sentences = []
targets = []
sentence = ['[CLS]']
target = [label_dict['-']]
for tup in tups:
  if (type(tup[0]) == float and tup[1] == 0) or tup[0] == '9999': # 문장 끝인 경우
    sentence.append("[SEP]")
    target.append(label_dict['-'])  
    sentences.append(sentence)
    targets.append(target)
    sentence = ['[CLS]']
    target = [label_dict['-']]
    continue
  if type(tup[0]) != float: # 원래 데이터는 있지만 특수기호 등으로 인해 nan이 들어간 경우는 제외
    sentence.append(tup[0])
    target.append(tup[1])

In [ ]:
sentences[:1]

[['[CLS]',
  '플랜티넷',
  '은',
  '2000년',
  '부터',
  '약',
  '20년',
  '동안',
  '축척한',
  '유하',
  '어',
  '디지털',
  '콘텐츠',
  '데이터베이스(DB)',
  '를',
  '바탕',
  '으로',
  '변화',
  '하',
  '는',
  '모바일(AW)',
  'PC',
  '환경',
  '에',
  '적합한',
  '새롭',
  '고',
  '혁신적',
  '이',
  'ㄴ',
  '필터링',
  '솔루션',
  '을',
  '연구',
  '하고',
  '제공',
  '하',
  'ㅂ니다',
  '.',
  '[SEP]']]

## 버트 인풋 생성

In [ ]:
from transformers import TFBertForMaskedLM
from transformers import BertTokenizer
from transformers import AutoTokenizer

# model = TFBertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
# tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
# model = TFBertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/812449f1a6bc736e693db7aa0e513e5e90795a62/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/812449f1a6bc736e693db7aa0e513e5e90795a62/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/812449f1a6bc736e693db7aa0e513e5e90795a62/tokenizer_config.json


Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/812449f1a6bc736e693db7aa0e513e5e90795a62/config.json
Model config BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}



In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels):
  tokenized_sentence = []
  labels = []

  for word, label in zip(sentence, text_labels):
    tokenized_word = tokenizer.tokenize(word)
    n_subwords = len(tokenized_word)

    tokenized_sentence.extend(tokenized_word)
    labels.extend([label] * n_subwords)

  return tokenized_sentence, labels

In [ ]:
tokenized_texts_and_labels = [tokenize_and_preserve_labels(sent, labs)
                              for sent, labs in zip(sentences, targets)]

In [ ]:
print(tokenized_texts_and_labels[:2]) # [(문장, 개체들), (문장, 개체들), ...]

[(['[CLS]', '플랜', '##티', '##넷', '은', '2000', '##년', '부터', '약', '20', '##년', '동안', '축', '##척', '##한', '유', '##하', '어', '디지털', '콘텐츠', '데이터베이스', '(', 'DB', ')', '를', '바탕', '으로', '변화', '하', '는', '모바일', '(', 'A', '##W', ')', 'PC', '환경', '에', '적합', '##한', '새롭', '고', '혁신', '##적', '이', 'ㄴ', '필터', '##링', '솔루션', '을', '연구', '하고', '제공', '하', 'ㅂ', '##니다', '.', '[SEP]'], [21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 7, 7, 7, 7, 7, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 21, 21, 21, 21, 21, 21, 21]), (['[CLS]', '주요', '##업', '##무', '[SEP]'], [21, 21, 21, 21, 21])]


In [ ]:
# [문장, 문장, ...]
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
# [개체들, 개체들, ...]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [ ]:
print(np.quantile(np.array([len(x) for x in tokenized_texts]), 0.975))

80.0


In [ ]:
max_len = 80

# 데이터 패딩
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype = "int", value=tokenizer.convert_tokens_to_ids("[PAD]"), truncating="post", padding="post")

In [ ]:
# 태그 패딩
tags = pad_sequences([lab for lab in labels], maxlen=max_len, value=label_dict["[PAD]"], padding='post',\
                     dtype='int', truncating='post')

In [ ]:
input_ids[0]

array([    2,  7942,  2747,  2881,  1497,  4470,  2440,  3631,  1397,
        3619,  2440,  3764,  1673,  2135,  2470,  1490,  2205,  1406,
        5476,  5156, 15626,    12, 16731,    13,  1022,  4875,  3603,
        3908,  1889,   793,  5221,    12,    37,  2350,    13,  6380,
        3828,  1421,  6273,  2470,  5415,   594,  4428,  2125,  1504,
         183, 12047,  2557,  8463,  1498,  3692,  6159,  3894,  1889,
         187,  3606,    18,     3,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0])

In [ ]:
tags[0]

array([21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
       21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,  7,  7,  7,  7,
        7, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       21, 21, 21, 21, 21, 21, 21, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
       25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25])

In [ ]:
# 어텐션 마스크
attention_masks = np.array([[int(i != tokenizer.convert_tokens_to_ids("[PAD]")) for i in ii] for ii in input_ids])

In [ ]:
# train 데이터에서 10%를 validation 데이터로 분리
#tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, random_state=2018, test_size=0.1)
#tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=2018, test_size=0.1)

In [ ]:
# 8:1:1로 분리
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, random_state=2018, test_size=0.2)
val_inputs, test_inputs, val_tags, test_tags = train_test_split(val_inputs, val_tags, random_state=2018, test_size=0.5)

tr_masks, val_masks = train_test_split(attention_masks, random_state=2018, test_size=0.2)
val_masks, test_masks = train_test_split(val_masks, random_state=2018, test_size=0.5)

In [ ]:
# TPU 작동을 위해 실행
#resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
#tf.config.experimental_connect_to_cluster(resolver)
#tf.tpu.experimental.initialize_tpu_system(resolver)

## 모델 생성

### Import

In [ ]:
#!pip install git+https://www.github.com/keras-team/keras-contrib.git
#!pip install keras-crf
#from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
#from keras_crf import CRFModel
#from keras.models import Sequential

In [ ]:
!pip install keras-bert
!pip install bert-for-tf2
#!pip install tr2crf
!pip install git+https://www.github.com/keras-team/keras-contrib.git
#!pip install tensorflow==2.3.0
#!pip install --upgrade tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33516 sha256=3fde243962309ffb67541e31688bf05a4ad0e4a9c6e9954c439461c3ebd85c56
  Stored in directory: /root/.cache/pip/wheels/a1/34/ed/6bbd71716d7bcea30d75e8bc5aeb94f4cb52636295c8343534
  Created wheel for keras-transformer: filename=keras_transformer-0.40.0-py3-none-any.whl size=12304 sha256=907ded9a62e5c98c2e26e65e1901ab163f92a80e9359e3ef6c03a23309e80b10
  Stored in directory: /root/.cache/pip/wheels/07/cd/a7/a8fa93f7e177eee0101fed63179f7a2fa3b53671ffaad82bfd
  Created wheel for keras-embed-sim: filename=keras_embed_sim-0.10.0-py3-none-any.whl size=3959 sha256=eff62ef785ea28ef5819bbf9b0140dab851daa68f94aa54e896b8051055d406f
  Stored in directory: /root/.cache/pip/wheels/59/bd/9a/ec6e575aaa50687d7af968bde7ce710b542eeaa9ee7978d4ba
  Created wheel for keras-layer-normalization: filename=keras_layer_normalizatio

In [ ]:
import tensorflow_hub as hub
import keras
from keras.models import Model, Sequential
from tensorflow.keras.layers import Input
from keras.layers import GRU, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda, BatchNormalization, Add
from keras.utils import to_categorical
from keras.callbacks import CSVLogger
from keras.optimizers import Adam
from keras_bert import load_trained_model_from_checkpoint
from keras_contrib.layers import CRF
from transformers import TFBertForMaskedLM, TFBertModel, TFAutoModelForSequenceClassification
import bert

### 삽질한거

In [ ]:
## 1차 시도
SEQ_LEN = max_len
def create_model():
  model = TFBertModel.from_pretrained("bert-base-multilingual-cased", from_pt=True, num_labels=len(label_dict), output_attentions = False,
    output_hidden_states = False)

  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids') # 토큰 인풋
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks') # 마스크 인풋

  bert_outputs = model([token_inputs, mask_inputs])
  bert_outputs = bert_outputs[0] # shape : (Batch_size, max_len, 30(개체의 총 개수))
  nr = tf.keras.layers.Dense(30, activation='softmax')(bert_outputs) # shape : (Batch_size, max_len, 30)
  
  nr_model = Sequential()
  #nr_model = tf.keras.Model([token_inputs, mask_inputs], nr)
  nr_model.add(tf.keras.Model([token_inputs, mask_inputs], nr))
  
  nr_model.add(Bidirectional(LSTM(128))) # 128: hidden units
  #nr_model.add(Dense(50, activation="relu"))
  nr_model.add(CRF(26)) # 26: tag size

  nr_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=['sparse_categorical_accuracy'])
  nr_model.summary()
  return nr_model

In [ ]:
## 2차 시도
from tensorflow.keras import Model
from keras_contrib.layers import CRF

SEQ_LEN = max_len
embedding_dim = 128
hidden_units = 64
dropout_ratio = 0.3
tag_size = 26
vocab_size = 119547 + 1

def create_model():
  model = TFBertModel.from_pretrained("bert-base-multilingual-cased", from_pt=True, num_labels=len(label_dict), output_attentions = False,
    output_hidden_states = False)

  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids') # 토큰 인풋
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks') # 마스크 인풋
  print("aaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
  bert_outputs = model([token_inputs, mask_inputs])
  print("bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb")
  last_encoder_layer = bert_outputs[0]
  print("ccccccccccccccccccccccccccccccccccccccc")
  last_encoder_layer = Dropout(dropout_ratio)(last_encoder_layer)
  print("ccccccccccccccccccccccccccccccccccccccc")
  model_bilstm = Bidirectional(LSTM(units=hidden_units, return_sequences=True))(last_encoder_layer)
  print("ddddddddddddddddddddddddddddddddddddddddd")
  model_dense = TimeDistributed(Dense(tag_size, activation='relu'))(model_bilstm)
  print("eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee")
  #base = Model(inputs=token_inputs, outputs=model_dense)
  print("fffffffffffffffffffffffffffffffffffffff")
  model = CRFModel(model_dense, tag_size)
  print("fffffffffffffffffffffffffffffffffffffff")
  nr_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00002), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=['sparse_categorical_accuracy'])
  nr_model.summary()
  return nr_model

In [ ]:
# 3차 시도
#!pip install git+https://www.github.com/keras-team/keras-contrib.git
#from keras.models import Sequential
#from keras.layers import InputLayer
#from keras_contrib.layers import CRF
import torch.nn as nn
import torch
from torchcrf import CRF

SEQ_LEN = max_len
embedding_dim = 128
hidden_units = 64
dropout_ratio = 0.3
tag_size = 26
vocab_size = 119547 + 1

def create_model():
  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids') # 토큰 인풋
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks') # 마스크 인풋
  print("aaa")
  print("bbb")
  bert_output = TFBertModel.from_pretrained("bert-base-multilingual-cased", from_pt=True, num_labels=len(label_dict), output_attentions = False,
    output_hidden_states = False)([token_inputs, mask_inputs])
  bert_output = bert_output[0] 
  bert_output = nn.Dropout(dropout_ratio)(bert_output)
  print("ccc")
  bilstm_output, hc = nn.LSTM(hidden_units, (hidden_units), bidirectional=True)(bert_output)
  logits = nn.Linear(hidden_units, tag_size)(bilstm_output)
  crf = CRF(tag_size, batch_first=True)
  log_likelihood, tags = crf(logits, label), crf.decode(nn.Linear(hidden_units, len(label_dict)))
  print("fffffffffffffffffffffffffffffffffffffff")
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00002), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=['sparse_categorical_accuracy'])
  model.summary()
  return model

In [ ]:
# 4차 시도
input_length = max_len
hidden_size = 100
output_size = len(label_dict)
learning_rate = 0.001

from keras.layers.core import Reshape

def create_model():
  input_ids = Input(shape=(input_length,), name='input_word_ids', dtype=tf.int32)
  attention_mask = Input(shape=(input_length,), name='attention_mask', dtype=tf.float32)
  #input_ids = tensor_util.convert_nonref_to_tensor(input_ids, dtype_hint=tf.int32, name='input_word_ids')
  #attention_mask = tensor_util.convert_nonref_to_tensor(attention_mask, dtype_hint=tf.float32, name='attention_mask')
  
  bert_model = TFBertModel.from_pretrained("bert-base-multilingual-cased", num_labels=output_size)([input_ids, attention_mask])
  #bert_model = TFBertModel.from_pretrained("klue/roberta-small", from_pt=True, num_labels=len(label_dict), output_attentions = False,
  #   output_hidden_states = False)([input_ids, attention_mask])
  #bert_model = TFBertModel.from_pretrained("bert-base-multilingual-cased", from_pt=True, num_labels=len(label_dict), output_attentions = False,
  #   output_hidden_states = False)(input_ids, attention_mask=attention_mask)
  #bert_model = TFBertModel.from_pretrained("bert-base-multilingual-cased", from_pt=True, num_labels=len(label_dict), output_attentions = False,
  #   output_hidden_states = False)
  #bert_model.save_pretrained("model/")
  #bert_model = TFBertForMaskedLM.from_pretrained("model/")
  #bert_output = bert_model(input_ids, attention_mask=attention_mask)[0]
  bert_output = bert_model[0]

  #bilstm_layer = Bidirectional(LSTM(hidden_size, return_sequences=True))
  #bilstm_layer_out = bilstm_layer(Dropout(0.5)(bert_output))

  #time_distributed_layer = TimeDistributed(Dense(output_size))
  #time_distributed_layer_out = time_distributed_layer(bilstm_layer_out)

  #crf = CRF(dtype=tf.float32, units=output_size)
  #crf_out = crf(BatchNormalization()(bilstm_layer_out))

  model = Model(inputs=[input_ids, attention_mask], outputs=bert_output)
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
      loss=keras.losses.BinaryCrossentropy(from_logits=True),
      metrics=keras.metrics.BinaryAccuracy(threshold=0.0))
      # loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      # metrics=['sparse_categorical_accuracy'])
  model.summary()
  return model

### 최근 시도

In [ ]:
from transformers import BertConfig, BertModel, AutoModelForSequenceClassification
from transformers import TFAutoModel
from tensorflow_probability.python.internal import tensor_util

In [ ]:
print(np.shape(tr_inputs), np.shape(tr_masks), np.shape(tr_tags))
print(np.shape(val_inputs), np.shape(val_masks), np.shape(val_tags))
print(np.shape(test_inputs), np.shape(test_masks), np.shape(test_tags))

(2565, 80) (2565, 80) (2565, 80)
(321, 80) (321, 80) (321, 80)
(321, 80) (321, 80) (321, 80)


# Bert

In [ ]:
# 5차 시도
# bert
SEQ_LEN = max_len
input_length = max_len
hidden_size = 100
output_size = len(label_dict)
learning_rate = 0.00002

def create_model():
  model = TFBertModel.from_pretrained("klue/bert-base", from_pt=True, num_labels=len(label_dict), output_attentions = False,
    output_hidden_states = False)

  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids') # 토큰 인풋
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks') # 마스크 인풋

  bert_outputs = model([token_inputs, mask_inputs])
  bert_outputs = bert_outputs[0]
  nr = tf.keras.layers.Dense(30, activation='softmax')(bert_outputs)

  #bilstm_layer = Bidirectional(LSTM(hidden_size, return_sequences=True))
  #bilstm_layer_out = bilstm_layer(nr)

  #crf = CRF(dtype=tf.float32, units=output_size)
  #crf_out = crf(bilstm_layer_out)
  
  nr_model = tf.keras.Model([token_inputs, mask_inputs], nr)
  
  nr_model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=['sparse_categorical_accuracy'])
  nr_model.summary()
  return nr_model

In [ ]:
!pip list

In [ ]:
from transformers import logging
logging.set_verbosity_error()

# GPU 사용하여 훈련
bs = 64
nr_model = create_model()
nr_model.fit([tr_inputs, tr_masks], tr_tags, validation_data=([val_inputs, val_masks], val_tags), epochs=50, shuffle=False, batch_size=bs)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 80)]         0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 80)]         0           []                               
                                                                                                  
 tf_bert_model_4 (TFBertModel)  TFBaseModelOutputWi  110617344   ['input_word_ids[0][0]',         
                                thPoolingAndCrossAt               'input_masks[0][0]']            
                                tentions(last_hidde                                               
                                n_state=(None, 80,                                          

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


41/41 [==============================] - 23s 197ms/step - loss: 0.5234 - sparse_categorical_accuracy: 0.8921 - val_loss: 0.1975 - val_sparse_categorical_accuracy: 0.9646
Epoch 2/50
41/41 [==============================] - 5s 125ms/step - loss: 0.1490 - sparse_categorical_accuracy: 0.9708 - val_loss: 0.1523 - val_sparse_categorical_accuracy: 0.9667
Epoch 3/50
41/41 [==============================] - 5s 125ms/step - loss: 0.1225 - sparse_categorical_accuracy: 0.9727 - val_loss: 0.1345 - val_sparse_categorical_accuracy: 0.9677
Epoch 4/50
41/41 [==============================] - 5s 125ms/step - loss: 0.1066 - sparse_categorical_accuracy: 0.9740 - val_loss: 0.1229 - val_sparse_categorical_accuracy: 0.9697
Epoch 5/50
41/41 [==============================] - 5s 126ms/step - loss: 0.0946 - sparse_categorical_accuracy: 0.9754 - val_loss: 0.1162 - val_sparse_categorical_accuracy: 0.9708
Epoch 6/50
41/41 [==============================] - 5s 126ms/step - loss: 0.0850 - sparse_categorical_accuracy

In [ ]:
# # TPU로 훈련
# bs = 32
# strategy = tf.distribute.experimental.TPUStrategy(resolver)
# # TPU를 활용하기 위해 context로 묶어주기
# with strategy.scope():
#   nr_model = create_model()
#   nr_model.fit([tr_inputs, tr_masks], tr_tags, validation_data=([val_inputs, val_masks], val_tags), epochs=10, shuffle=False, batch_size=bs)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

y_predicted = nr_model.predict([test_inputs, test_masks])

11/11 [==============================] - 3s 34ms/step


In [ ]:
f_label = [i for i, j in label_dict.items()]
test_tags_l = [index_to_ner[x] for x in np.ravel(test_tags).astype(int).tolist() if x < 26]
y_predicted_l = [index_to_ner[x] for x in np.ravel(np.argmax(y_predicted, axis=2)).astype(int).tolist() if x < 26]
f_label.remove("[PAD]")

In [ ]:
# f1 score 측정
# macro avg: 전체 정답을 기준으로 f1 score 측정
# macro avg: 각 개체별 f1 score 가중평균
print(classification_report(test_tags_l, y_predicted_l, labels=f_label))

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

              precision    recall  f1-score   support

         nan       0.00      0.00      0.00        21
        B_DB       0.50      1.00      0.67         1
        I_PL       0.00      0.00      0.00         0
        I_CS       0.36      0.37      0.37        73
        B_DT       0.50      0.71      0.59        28
        B_CS       0.74      0.76      0.75       113
        I_HW       1.00      0.14      0.25         7
        B_AW       0.61      0.57      0.59        47
        I_SC       0.00      0.00      0.00         2
        B_SS       0.49      0.47      0.48        73
        B_PL       0.56      0.56      0.56         9
        I_NT       0.00      0.00      0.00         2
        9999       0.98      1.00      0.99       180
        I_AW       0.27      0.14      0.19        49
        B_SC       0.00      0.00      0.00         3
        I_OS       0.00      0.00      0.00         0
        I_DT       0.40      0.44      0.42        52
        I_SW       0.42    

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
def ner_inference(test_sentence):
  

  tokenized_sentence = np.array([tokenizer.encode(test_sentence, max_length=max_len, truncation=True, padding='max_length')])
  tokenized_mask = np.array([[int(x!=1) for x in tokenized_sentence[0].tolist()]])
  ans = nr_model.predict([tokenized_sentence, tokenized_mask])
  ans = np.argmax(ans, axis=2)

  tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])
  new_tokens, new_labels = [], []
  for token, label_idx in zip(tokens, ans[0]):
    if (token.startswith("##")):
      new_labels.append(index_to_ner[label_idx])
      new_tokens.append(token[2:])
    elif (token=='[CLS]'):
      pass
    elif (token=='[SEP]'):
      pass
    elif (token=='[PAD]'):
      pass
    elif (token != '[CLS]' or token != '[SEP]'):
      new_tokens.append(token)
      new_labels.append(index_to_ner[label_idx])

  for token, label in zip(new_tokens, new_labels):
      print("{}\t{}".format(label, token))
     

# Bert + Bilstm

In [ ]:
# 5차 시도
# bert + bilstm
SEQ_LEN = max_len
input_length = max_len
hidden_size = 100
output_size = len(label_dict)
learning_rate = 0.001

def create_model():
  model = TFBertModel.from_pretrained("klue/bert-base", from_pt=True, num_labels=len(label_dict), output_attentions = False,
    output_hidden_states = False)

  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids') # 토큰 인풋
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks') # 마스크 인풋

  bert_outputs = model([token_inputs, mask_inputs])
  bert_outputs = bert_outputs[0] # shape : (Batch_size, max_len, 30(개체의 총 개수))
  nr = tf.keras.layers.Dense(30, activation='softmax')(bert_outputs) # shape : (Batch_size, max_len, 30)

  bilstm_layer = Bidirectional(LSTM(hidden_size, return_sequences=True))
  bilstm_layer_out = bilstm_layer(nr)

  #crf = CRF(dtype=tf.float32, units=output_size)
  #crf_out = crf(bilstm_layer_out)
  
  nr_model_b = tf.keras.Model([token_inputs, mask_inputs], bilstm_layer_out)
  
  nr_model_b.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0002), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=['sparse_categorical_accuracy'])
  nr_model_b.summary()
  return nr_model_b

In [ ]:
# GPU 사용하여 훈련
bs = 32
nr_model_b = create_model()
nr_model_b.fit([tr_inputs, tr_masks], tr_tags, validation_data=([val_inputs, val_masks], val_tags), epochs=3, shuffle=False, batch_size=bs)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 102)]        0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 102)]        0           []                               
                                                                                                  
 tf_bert_model_2 (TFBertModel)  TFBaseModelOutputWi  110617344   ['input_word_ids[0][0]',         
                                thPoolingAndCrossAt               'input_masks[0][0]']            
                                tentions(last_hidde                                               
                                n_state=(None, 102,                                         

11/81 [===>..........................] - ETA: 28:56 - loss: 4.3993 - sparse_categorical_accuracy: 0.5339

In [ ]:
# # TPU로 훈련
# bs = 32
# strategy = tf.distribute.experimental.TPUStrategy(resolver)
# # TPU를 활용하기 위해 context로 묶어주기
# with strategy.scope():
#   nr_model = create_model()
#   nr_model.fit([tr_inputs, tr_masks], tr_tags, validation_data=([val_inputs, val_masks], val_tags), epochs=10, shuffle=False, batch_size=bs)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

y_predicted = nr_model.predict([test_inputs, test_masks])

4/4 [==============================] - 12s 3s/step


In [ ]:
f_label = [i for i, j in label_dict.items()]
test_tags_l = [index_to_ner[x] for x in np.ravel(test_tags).astype(int).tolist() if x < 26]
y_predicted_l = [index_to_ner[x] for x in np.ravel(np.argmax(y_predicted, axis=2)).astype(int).tolist() if x < 26]
f_label.remove("[PAD]")

In [ ]:
# f1 score 측정
# macro avg: 전체 정답을 기준으로 f1 score 측정
# macro avg: 각 개체별 f1 score 가중평균
print(classification_report(val_tags_l, y_predicted_l, labels=f_label))

              precision    recall  f1-score   support

           -       0.98      0.93      0.95      3352
         nan       0.00      0.00      0.00         0
        B_SC       0.00      0.00      0.00         0
        I_SC       0.00      0.00      0.00         0
        B_CS       0.37      0.73      0.49        59
        I_CS       0.37      0.72      0.49        50
        B_AW       0.33      0.40      0.36        15
        I_AW       0.48      0.50      0.49        22
        B_SS       0.46      0.28      0.35        60
        I_SS       0.22      0.55      0.31        78
        B_DB       0.00      0.00      0.00         0
        B_PL       1.00      0.25      0.40         8
        I_PL       0.00      0.00      0.00         0
         I_S       0.00      0.00      0.00         0
        B_DT       0.80      0.57      0.67        21
        I_DT       0.80      0.50      0.62         8
        B_SW       0.00      0.00      0.00         2
        B_NT       0.00    

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

In [ ]:
def ner_inference(test_sentence):
  

  tokenized_sentence = np.array([tokenizer.encode(test_sentence, max_length=max_len, truncation=True, padding='max_length')])
  tokenized_mask = np.array([[int(x!=1) for x in tokenized_sentence[0].tolist()]])
  ans = nr_model.predict([tokenized_sentence, tokenized_mask])
  ans = np.argmax(ans, axis=2)

  tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])
  new_tokens, new_labels = [], []
  for token, label_idx in zip(tokens, ans[0]):
    if (token.startswith("##")):
      new_labels.append(index_to_ner[label_idx])
      new_tokens.append(token[2:])
    elif (token=='[CLS]'):
      pass
    elif (token=='[SEP]'):
      pass
    elif (token=='[PAD]'):
      pass
    elif (token != '[CLS]' or token != '[SEP]'):
      new_tokens.append(token)
      new_labels.append(index_to_ner[label_idx])

  for token, label in zip(new_tokens, new_labels):
      print("{}\t{}".format(label, token))
     